## 1. 路径最小距离累加和
* 二维数组matrix，一个人从左上角出发，到达右下角。沿途可以向下或向右，沿途遇到的数字累加就是距离累加和
* 返回最小距离累加和

In [6]:
#想象有个递归方法，从0,0出发到达x,y，返回最小值
def dp(arr):
    N = len(arr)
    M = len(arr[0])
    dp = [[0] * M for _ in range(N)]
    dp[0][0] = arr[0][0]
    for i in range(1, M):
        dp[0][i] = dp[0][i-1] + arr[0][i]
    for i in range(1, N):
        dp[i][0] = dp[i-1][0] + arr[i][0]
    for x in range(1, N):
        for y in range(1, M):
            dp[x][y] = min(dp[x-1][y], dp[x][y-1]) + arr[x][y]
    return dp[N-1][M-1]
def minDistance(arr):
    if not arr or not arr[0]:
        return 0
    ans = dp(arr)
    return ans

#优化空间
# i,j 只依赖 i-1,j 和 i,j-1. 只用一个数组足以完成更新

def dp2(arr):
    N = len(arr)
    M = len(arr[0])
    dp = [0] * M
    dp[0] = arr[0][0]
    # 首行
    for i in range(1, M):
        dp[i] = dp[i-1] + arr[0][i]
    
    for i in range(1, N):
        dp[0] += arr[i][0]
        for j in range(1, M):
            dp[j] = min(dp[j-1] , dp[j]) + arr[i][j]
    return dp[M-1]

#如果 i、j依赖于左上和上面的值，从右往左填，对于i,j，此时左上(i-1,j-1)、上(i-1,j)都没更新

#如果 i、j依赖于左上、上、左的三个值，从左往右填，左先更新，但更新时把旧值用一个变量记住。

#如果可以省数组，用短的宽度作为唯一数组，滚动更新

## 2.凑钱
* 给定一个货币数组，其中的值都是正数。再给定一个正数aim。
* 每个值都认为是一张货币，即使面值相同，也认为不同张。返回组成aim的方法数
* 例如arr = [1,1,1]，aim是2， 返回3(第0、1；第1、2；第0、2)


In [7]:
#从左往右模型
def process(arr, index, res):
    if rest < 0:
        return 0
    if index == len(arr):
        return 1 if rest == 0 else 0
    return process(arr, index+1, rest) + process(arr, index+1, res-arr[index])


def coinWays(arr, aim):
    return process(arr, 0, aim)


#从左往右模型
def pick(arr, i, j):
    if i < 0 or i > len(arr) - 1 or j < 0 or j > len(arr[0]):
        return 0
    else:
        return arr[i][j]
def dp(arr, aim):
    if aim == 0:
        return 1
    N = len(arr)
    dp = [[0] * (aim+1) for _ in range(N+1)]
    dp[N][0] = 1
    for index in range(N-1, -1, -1):
        for rest in range(aim+1):
            dp[index][rest] = pick(dp, index+1, rest) + pick(dp, index+1, res - arr[index])
    return dp[0][aim]


def coinWays(arr, aim):
    return process(arr, 0, aim)

## 2.凑钱(张数无限版本)
* 给定一个货币数组，其中的值都是正数。再给定一个正数aim。
* 每个值都认为是一张面值，提供无限张。返回组成aim的方法数
* 例如arr = [1,2]，aim是4， 返回3(1+1+1+1,1+1+2,2+2)

In [10]:
#arr[index...]所有的面值，每个面值任意选择，组成正好rest，返回方法数。
def process(arr, index, rest):
    if index == len(arr):
        return 1 if rest == 0 else 0
    ways = 0
    for nums in range(rest // arr[index] + 1): #nums<= rest // arr[index]
        ways += process(arr, index+1, rest - (nums * arr[index]))
    return ways

def coinWays(arr, aim):
    if not arr or aim < 0:
        return 0
    return process(arr, 0, aim)

In [13]:
def dp(arr, aim):
    if not arr or aim < 0:
        return 0
    N = len(arr)
    dp = [[0]*(aim+1) for _ in range(N+1)]
    dp[N][0] = 1
    for index in range(N-1,-1,-1):
        for rest in range(aim+1):
            ways = 0
            for nums in range(rest // arr[index] + 1): #nums<= rest // arr[index]
                ways += dp[index+1][rest - (nums * arr[index])]
                
            dp[index][rest] = ways
    
    return dp[0][aim]


def coinWays(arr, aim):
    if not arr or aim < 0:
        return 0
    return dp(arr, aim)

* 当每个格子只用O(1)时间计算出，则记忆化搜索和严格表结构同样效率
* 当每个格子需要for循环计算出，则严格表结构可以继续优化

In [12]:
'''
             # √
[i] ...e d c b a

假设想求   √位置，根据for循环可知需要计算a+b+c+d(...可能更多)。
但如果已知 #位置，则√位置只需要计算 # + a有限步数（如果已知只3次for循环，则减去e）

'''

In [15]:

def dp2(arr, aim):
    if not arr or aim < 0:
        return 0
    N = len(arr)
    dp = [[0]*(aim+1) for _ in range(N+1)]
    dp[N][0] = 1
    for index in range(N-1,-1,-1):
        for rest in range(aim+1):
            dp[index][rest] = dp[index+1][rest]
            if rest - arr[index] >= 0:
                dp[index][rest] += dp[index][rest - arr[index]] #省去for循环
    
    return dp[0][aim]


def coinWays(arr, aim):
    if not arr or aim < 0:
        return 0
    return dp(arr, aim)

## 3.凑钱(同面值无不同)
* 给定一个货币数组，其中的值都是正数。再给定一个正数aim。
* 每个值都认为是一张面值，同一面值的钱无不同。返回组成aim的方法数
* 例如arr = [1,2,1,1,2,1,2]，aim是4， 返回3(1+1+1+1,1+1+2,2+2)

In [16]:
#思路：将数组遍历，记录面值频次

In [18]:


def coinWays(arr, aim):
    count = dict()
    for value in arr:
        if value not in count:
            count[value] = 1
        else:
            count[value] += 1
    coins = list()
    nums = list()
    for (v, c) in count.items():
        coins.append(v)
        nums.append(c)
    return process(coins, nums, 0, aim)
# coins 面值数组，正数且去重
# nums 面值对应的张数
def process(coins, nums, index, rest):
    if index == len(coins):
        return 1 if rest == 0 else 0
    ways = 0
    for n in range( min(rest // coins[index], nums[index]) + 1 ):
        ways += process(coins, nums, index+1, rest - n*coins[index])
    return ways

In [19]:


def coinWays(arr, aim):
    count = dict()
    for value in arr:
        if value not in count:
            count[value] = 1
        else:
            count[value] += 1
    coins = list()
    nums = list()
    for (v, c) in count.items():
        coins.append(v)
        nums.append(c)
    return dp(coins, nums, 0, aim)
# coins 面值数组，正数且去重
# nums 面值对应的张数

def dp(coins, nums, aim):
    if not coins or not nums or aim < 0:
        return 0
    N = len(coins)
    dp = [[0]*(aim+1) for _ in range(N+1)]  
    dp[N][0] = 1
    for index in range(N-1, -1, -1):
        for rest in range(aim+1):
            dp[index][rest] = dp[index+1][rest]
            if rest - coins[index] >= 0:
                dp[index][rest] += dp[index][rest-coins[index]]
            if rest - coins[index] * (nums[index]+1) >= 0:
                dp[index][rest] -= dp[index+1][rest - coins[index] * (nums[index] + 1)]

    
    return dp[0][aim]

## 4.醉汉问题
给定5个参数，N、M、row、col，k
表示在N*M区域上，醉汉bob初始在(row,col)位置
bob一共要迈出k步，每步都等概率向上、下、左、右四个方向走一个单位。任何时候bob离开N*M区域，就立即死亡
返回k步之后，bob还活着的概率

In [21]:
# 如果N*M无限大，那k步的可能就是4^k种可能
# 计算出k步后bob在区域内的可能，除以上面4^k就是答案概率

In [23]:
def livePosibility(row, col, k, N, M):
    return process(row,col,k,N,M) * 1.0 / (4 ** k)

#目前在row,col位置，还剩rest步走，走完如果还在N*M中就获得1点生存，返回总的生存点数
def process(row,col,rest,N,M):
    if row < 0 or row == N or col < 0 or col == N:
        return 0 #死了
    #还在区域内
    if rest == 0:
        return 1
    
    up = process(row - 1, col, res -1, N, M)
    down = process(row + 1, col, res -1, N, M)
    left = process(row, col - 1, res -1, N, M)
    right = process(row, col+1, res -1, N, M)
    return up + down + left + right


In [24]:
def livePosibility2(row, col, k, N, M):
    return dp(row, col, k, N, M) * 1.0 / (4 ** k)

def pick(dp, row,col,rest,N,M):
    if row < 0 or row == N or col < 0 or col == M:
        return 0
    return dp[row][col][rest]

def dp(row, col, k, N, M):
    if row < 0 or row >= N or col < 0 or col >= N:
        return 0 #上来就死了，没有后续
    dp = [[[0] * (k+1) for _ in range(M)] for _ in range(N)]
    #k=0层，0<=col<M ==>1; 0<=row<N ==>1; 
    for x in range(0, N):
        for y in range(0, M):
            dp[x][y][0] = 1
    for rest in range(1, k+1):
        for x in range(0, N):
            #这里改成用pk，边界更保险。不能用range(M-1)，让M-1上无结果了就，实际上虽然不能往右走，但还可以上下左，3个值相加
            for y in range(0,M):
                up = pick(dp, row-1, col, rest -1)
                down = pick(dp, row+1, col, rest -1)
                left = pick(dp, row, col-1, rest -1)
                right = pick(dp, row, col+1, rest -1)
                dp[x][y][rest] = up+down+left+right
    return dp[row][col][k]